In [1]:
import pandas as pd

In [121]:
import re
import os
import glob
import json
import email
import time
import argparse
import smtplib, ssl # For mail
import pandas as pd

from itertools import groupby
from collections import defaultdict
from os.path import expanduser





class Utils:
    '''
    Useful generic utils
    '''
    @staticmethod
    def get_home_dir():
        '''
        '''
        return expanduser("~")

    @staticmethod
    def get_immediate_subdirectories(a_dir):
        '''
        https://stackoverflow.com/a/800201
        '''
        for name in os.listdir(a_dir):
            p = os.path.join(a_dir, name)
            
            if os.path.isdir(p):
                yield p 


class Mail:

    PASSWORD_PATH = '.gmail/settings.json'

    def __init__(self,):
        self.connect_to_gmail()

    @staticmethod
    def get_password():
        password_filename = os.path.join(
            Utils.get_home_dir(), 
            Mail.PASSWORD_PATH
        )

        with open(password_filename) as f:
            data = json.load(f)
        return data['password']

    def connect_to_gmail(self,):
        port = 587  # For starttls
        smtp_server = "smtp.gmail.com"
        sender_email = "alexandros.kanterakis@gmail.com"
        password = Mail.get_password() 

        context = ssl.create_default_context()
        self.server = smtplib.SMTP(smtp_server, port)

        self.server.ehlo()  # Can be omitted
        self.server.starttls(context=context)
        self.server.ehlo()  # Can be omitted
        self.server.login(sender_email, password)

        print ('CONNECTED TO GMAIL')



    def do_send_mail(self, to, subject, text, 
            sleep=10, 
            actually_send_mail=False,
            html=False,
        ):
        from email.header import Header
        from email.mime.text import MIMEText
        from email.mime.multipart import MIMEMultipart

        #msg = MIMEText(text, 'plain', 'utf-8') # If text is garbled, try this
        if html:
            msg = MIMEText(text, 'html')
        else:
            msg = MIMEText(text)

        sender_email = "alexandros.kanterakis@gmail.com"
        receiver_email = to


        email = MIMEMultipart('mixed') # email = MIMEMultipart()
        email['From'] = sender_email
        email['To'] = receiver_email
        email['Subject'] = Header(subject, 'utf-8')

        msg.set_payload(text.encode('utf-8')) #msg.set_payload(text.encode('ascii'))
        email.attach(msg)

        message = email.as_string()

        if False:
            message = 'Subject: {}\n\n{}'.format(subject, text)

        if actually_send_mail:
            self.server.sendmail(sender_email, receiver_email, message)
        else:
            print (text)
            #print (message)
        time.sleep(sleep)
        print ('Mail sent')

    def disconnect_from_gmail(self,):
        self.server.quit()
        print ('DISCONNECTED FROM GMAIL')



In [122]:
main  = Mail()

CONNECTED TO GMAIL


In [119]:
pattern = '''
<html>
  <body>
    Γεια σας, <br>
Παρακάτω ακολουθούν οι βαθμοί σας στο τελικό διαγώνισμα στο μάθημα:<br>
ΒΙΟΛ-109, Χρήσεις του Η/Υ και Βιολογικές Βάσεις δεδομένων.<br>

Ακολουθεί επίσης και η συνολική βαθμολογία σας. <br><br>

Όνομα: <b>{name}</b><br>
ΑΜ: <b>{AM}</b><br>
<pre>
{grades_str}
</pre>

Ο τελικός βαθμός σας στο μάθημα είναι: <strong>{final3_grade}</strong><br>
To ποσοστό των φοιτητών που έγραψαν καλύτερα από εσάς είναι: {perc:.2f}%<br>

Για απορίες παρακαλώ επικοινωνήστε μέσω του slack.<br>
Χαιρετώ,<br>
Αλέξανδρος Καντεράκης <br>
    
  </body>
</html>

'''

In [ ]:
df = pd.read_csv('Grades_109_2021_2022.csv')
df.rename(
    columns={
        'Unnamed: 0': 'names',
        'Unnamed: 1': 'AM',
        'Ασκήσεις': 'Ex1',
        'Unnamed: 3': 'Ex2',
        'Τελικό': 'Fin1',
        'Unnamed: 10': 'Fin2',
        'Unnamed: 11': 'Fin3',
        'Unnamed: 12': 'Fin4',
        'Unnamed: 13': 'Fin5',
        'Unnamed: 14': 'Fin6',
        'Unnamed: 15': 'Fin7',
        'Unnamed: 16': 'Fin8',
        'Unnamed: 17': 'Fin9',
        'Unnamed: 18': 'Fin10',
        'Unnamed: 19': 'Fin11',
        'Unnamed: 20': 'Fin12',


    }, 
    inplace=True
)

all_grades = []
for r in df.itertuples():
    
    
    name = r.names
    if name == 'Όνομα':
        continue
    AM = r.AM
    print (name, AM)
    ask1 = r.Ex1
    if pd.isna(ask1):
        ask1 = 0.0
        
    ask2 = r.Ex2
    if pd.isna(ask2):
        ask2 = 0.0
    
    final = {x : getattr(r, f'Fin{x}') for x in range(1,13)}
    final = {k : v if not pd.isna(v) else 0.0 for k,v in final.items()}
    
    #final_grade = 
    
    new_df = {
        'Άσκηση/Εξέταση': [],
        'Βαθμός':[],
    }
    
    new_df['Άσκηση/Εξέταση'].append('1η Άσκηση (20%)')
    new_df['Βαθμός'].append(ask1)
    
    new_df['Άσκηση/Εξέταση'].append('2η Άσκηση (20%)')
    new_df['Βαθμός'].append(ask2)
    
    s = 0
    for k,v in final.items():
        new_df['Άσκηση/Εξέταση'].append(f'Τελικό {k}η ερώτηση')
        
        if k==6:
            new_df['Βαθμός'].append(f'{v}/11')
            s += v/11
        else:
            new_df['Βαθμός'].append(f'{v}/10')
            s += v/10
            
    final1_grade = 10*s/12
        
    new_df['Άσκηση/Εξέταση'].append('Βαθμός Τελικού (60%)')
    new_df['Βαθμός'].append(final1_grade)
    
    final2_grade = 0.2*float(ask1) + 0.2*float(ask2) + 0.6*final1_grade
    all_grades.append(final2_grade)
    new_df['Άσκηση/Εξέταση'].append('Δεκαδικός Βαθμός Μαθήματος')
    new_df['Βαθμός'].append(final2_grade)

    final3_grade = round(final2_grade*2,0)/2
    new_df['Άσκηση/Εξέταση'].append('Βαθμός Μαθήματος')
    new_df['Βαθμός'].append(final3_grade)
   

    new_df = pd.DataFrame(new_df)
    grades_str = new_df.to_string(index=False)
    grades_str = (
        grades_str
        .replace('Άσκηση/Εξέταση', '<b>Άσκηση/Εξέταση</b>')
        .replace('Βαθμός', '<b>Βαθμός</b>', 1)
    )
    
    #grades_str = new_df.to_html(index=False)
    perc = 100*sum(x>final2_grade for x in all_grades2)/len(all_grades2)

    body = pattern.format(
        name=name,
        AM=AM,
        grades_str=grades_str,
        final3_grade=final3_grade,
        perc=perc,
    )
    
    subject = 'ΒΙΟΛ-109: Αποτελέσματα εξέτασης Ιανουαρίου και τελικός βαθμός στο μάθημα'
    mail = f'bio{AM}@edu.biology.uoc.gr'
    print (mail, final3_grade)
    #print (body)
    
    if True:
        main.do_send_mail(
            #to='alexandros.kanterakis@gmail.com',
            to=mail,
            subject=subject,
            text=body, 
            sleep=10, 
            actually_send_mail=True,
            html=True
        )
    
    
    #break

